In [13]:
from mobgap.data import LabExampleDataset

data_ms = LabExampleDataset().get_subset(cohort="MS")
long_test_ms = data_ms.get_subset(test="Test11")[0]
long_test_ms

,cohort,participant_id,time_measure,test,trial
0,MS,001,TimeMeasure1,Test11,Trial1


In [2]:
from mobgap.data import get_example_csv_data_path
path = get_example_csv_data_path()
all_data_files = sorted(list(path.rglob("*.csv")))
all_data_files

import pandas as pd
# assume that have a valid CSV.
data = pd.read_csv(all_data_files[0]) # this method right here changes it to Pandas form.
# data.head()

sampling_rate_hz = 100 # should just know this

# convert the csv into a dataframe
from pathlib import Path
from mobgap.consts import GRAV_MS2

def load_data(path: Path) -> pd.DataFrame:
    data = pd.read_csv(path)
    # convert acceleration g to m/s^2
    data[["acc_x", "acc_y", "acc_z"]] = (data[["acc_x", "acc_y", "acc_z"]] * GRAV_MS2)
    return data

# this CSV in partciualr had to get the ACC transformed into the correct units.
data = load_data(all_data_files[0])
data.head()

# participant metadata only requires sensor height (m) + cohort
participant_metadata = {
    'cohort': 'MS',
    'sensor_height_m': 0.964, # user input
    'height_m': 1.59
}

recording_metadata = {"measurement_condition": "laboratory"}

# at this point, should have the CSV of data in a valid data frame
# + participant metadata and recording metadata.

# only need to load in one recording into a GaitDataset.
loaded_data = {} # dictionary (first level: identifiers) of dictionaries {"SensorLabel": DataInDataFrame}
participant_metadata_dataset = {}
recording_metadata_dataset = {}

# identifying information doesn't matter because only one recording (indexing from 1 here)
loaded_data[("MS", "1")] = {
    "LowerBack": data
}
participant_metadata_dataset[("MS", "1")] = participant_metadata
recording_metadata_dataset[("MS", "1")] = recording_metadata

from mobgap.data import GaitDatasetFromData
dataset = GaitDatasetFromData(
    loaded_data,
    sampling_rate_hz,
    participant_metadata_dataset,
    recording_metadata_dataset,
    index_cols = ["cohort", "participant_id"]
)

focus_record = dataset[0]
focus_record.data_ss.head()
focus_record.participant_metadata
focus_record.recording_metadata


# focus_record = dataset[0] # a single record is of type GaitDatasetFromData
# focus_record.data_ss.head()
# type(focus_record)
# focus_record.recording_metadata

from mobgap.pipeline import MobilisedPipelineImpaired
pipeline_ms = MobilisedPipelineImpaired().run(focus_record)
pipeline_ms.per_wb_parameters_

# pipeline_ms = pipeline_ms.safe_run(focus_record)

# focus_record.participant_metadata
# focus_record.data_ss.head()



[PosixPath('/Users/georgehum/Library/Caches/mobgap/v0.10.0/data_csv/HA/001/TimeMeasure1_Test11_Trial1.csv'), PosixPath('/Users/georgehum/Library/Caches/mobgap/v0.10.0/data_csv/HA/001/TimeMeasure1_Test5_Trial1.csv'), PosixPath('/Users/georgehum/Library/Caches/mobgap/v0.10.0/data_csv/HA/001/TimeMeasure1_Test5_Trial2.csv'), PosixPath('/Users/georgehum/Library/Caches/mobgap/v0.10.0/data_csv/HA/002/TimeMeasure1_Test11_Trial1.csv'), PosixPath('/Users/georgehum/Library/Caches/mobgap/v0.10.0/data_csv/HA/002/TimeMeasure1_Test5_Trial1.csv'), PosixPath('/Users/georgehum/Library/Caches/mobgap/v0.10.0/data_csv/HA/002/TimeMeasure1_Test5_Trial2.csv'), PosixPath('/Users/georgehum/Library/Caches/mobgap/v0.10.0/data_csv/MS/001/TimeMeasure1_Test11_Trial1.csv'), PosixPath('/Users/georgehum/Library/Caches/mobgap/v0.10.0/data_csv/MS/001/TimeMeasure1_Test5_Trial1.csv'), PosixPath('/Users/georgehum/Library/Caches/mobgap/v0.10.0/data_csv/MS/001/TimeMeasure1_Test5_Trial2.csv'), PosixPath('/Users/georgehum/Libra

,start,end,n_strides,rule_name,rule_obj,duration_s,stride_duration_s,cadence_spm,stride_length_m,walking_speed_mps
wb_id,,,,,,,,,,
0,532,1040,7,max_break,"MaxBreakCriteria(consider_end_as_break=True, m...",5.08,0.887143,99.025362,1.054044,0.901577
1,3855,5298,18,max_break,"MaxBreakCriteria(consider_end_as_break=True, m...",14.43,1.353333,93.553641,1.038665,0.799753
2,7739,8679,10,max_break,"MaxBreakCriteria(consider_end_as_break=True, m...",9.40,1.034000,96.277083,0.958272,0.762278
3,9529,9932,5,max_break,"MaxBreakCriteria(consider_end_as_break=True, m...",4.03,1.332000,90.001462,1.071972,0.842267
4,11930,13602,16,max_break,"MaxBreakCriteria(consider_end_as_break=True, m...",16.72,1.303125,94.144852,1.070675,0.865277
